## Transfer learning based on different Neural Networks together with Logistic Regression

The main idea here is to try out different Neural Networks to feed into the logistic Regression Classifier.

In [1]:
# plotting imports and setup
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
matplotlib.rcParams['figure.figsize'] = [10,10]

First we have to import the dataset.

In [2]:
from keras.preprocessing import image
from os import listdir
from keras.applications.vgg19 import preprocess_input

fdir='Data/CERTH_ImageBlurDataset/TrainingSet/Naturally-Blurred/'
files= listdir(fdir)
X=[] #feature vector
images=[]
Y=[] #class vector (1='blurred', 0='in focus')
for fn in files:
    img_path = fdir+fn
    x=image.load_img(img_path, target_size=(224, 224))
    images.append(x)
    x=image.img_to_array(x)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    X.append(x)
    Y.append(1)
    
fdir='Data/CERTH_ImageBlurDataset/TrainingSet/Undistorted/'
files= listdir(fdir)
for fn in files:
    img_path = fdir+fn
    x=image.load_img(img_path, target_size=(224, 224))
    images.append(x)
    x=image.img_to_array(x)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    X.append(x)
    Y.append(0)

/Users/albert/miniconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
X_train = X
y_train = Y

In [4]:
import pandas as pd
fn='Data/CERTH_ImageBlurDataset/EvaluationSet/NaturalBlurSet.xlsx'
xl = np.array(pd.read_excel(fn))
val={}
for xx in xl:
    val[xx[0]]=xx[1]

In [5]:
fdir='Data/CERTH_ImageBlurDataset/EvaluationSet/NaturalBlurSet/'
files= listdir(fdir)
X_test=[] #feature vector
images=[]
y_test=[] #class vector (1='blurred', 0='in focus')
for fn in files:
    img_path = fdir+fn
    x=image.load_img(img_path, target_size=(224, 224))
    images.append(x)
    x=image.img_to_array(x)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    X_test.append(x)
    y_test.append((val[fn[:-4]]+1)/2)

### Using VGG19 for feature extraction

In [7]:
from keras import applications

model = applications.VGG19(include_top=False,weights='imagenet')

X_train_ = []
for xx in X_train:
    X_train_.append( model.predict(xx) )
    
X_test_ = []
for xx in X_test:
    X_test_.append( model.predict(xx) )

80150528/80134624 [==============================] - 10s 0us/step


In [8]:
reshape=np.shape(X_train_)[1]*np.shape(X_train_)[2]*np.shape(X_train_)[3]*np.shape(X_train_)[4]
X_train__=np.array(X_train_).reshape(len(X_train_),reshape)
X_test__=np.array(X_test_).reshape(len(X_test_),reshape)

#### optimizing the hyperparameters of the Logistic Regression Classifier


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from sklearn.linear_model import LogisticRegression

param_grid = {'C':  sp_uniform(0,10), 
              'intercept_scaling': sp_uniform(0,10),
             'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
              'tol': sp_uniform(0,0.01)
             }

opt_grid_lr = {}

grid = RandomizedSearchCV(LogisticRegression(),  param_distributions=param_grid,n_iter=100, cv=2)
grid.fit(X_train__, y_train)
opt_grid_lr=grid.best_params_

lr = LogisticRegression(**opt_grid_lr).fit(X_train__,y_train)
print 'LogisticRegression stats'
print 'training score: ', lr.score(X_train__,y_train)
print 'test score: ',lr.score(X_test__,y_test)
print("best parameters: {}".format(opt_grid_lr))

/Users/albert/miniconda2/lib/python2.7/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
